# Baseline model
As a baseline, we will implement a collaborative filtering recommendation algorithm to recommend a bot lane duo, given a particular matchup.

In [4]:
import pandas as pd

dataset = pd.read_csv('dataset.csv')

dataset.head()

,id,matchid,teamid,player,championid,position,kills,deaths,assists,killingsprees,...,win,goldearned,champlvl,totheal,totdmgtaken,towerkills,inhibkills_y,baronkills,dragonkills,harrykills
0,10,10,100,2,267,SUPP,0,2,12,0,...,0,9496,14,11707,17769,5,0,0,0,0
1,11,10,100,3,119,ADC,7,8,5,1,...,0,13136,14,2283,25627,5,0,0,0,0
2,17,10,200,9,222,ADC,15,3,9,2,...,1,15970,16,2802,17655,10,3,1,3,1
3,18,10,200,10,161,SUPP,4,5,19,1,...,1,12978,16,3242,13443,10,3,1,3,1
4,19,11,100,1,115,SUPP,2,7,5,0,...,0,7792,12,104,11576,2,0,0,0,0


In [5]:
dataset.dtypes

id                int64
matchid           int64
teamid            int64
player            int64
championid        int64
position         object
kills             int64
deaths            int64
assists           int64
killingsprees     int64
totdmgdealt       int64
visionscore       int64
win               int64
goldearned        int64
champlvl          int64
totheal           int64
totdmgtaken       int64
towerkills        int64
inhibkills_y      int64
baronkills        int64
dragonkills       int64
harrykills        int64
dtype: object

In [6]:
from sklearn.model_selection import train_test_split

# Step 1: Group data by match ID
match_ids = dataset['matchid'].unique()

# Step 2: Split match IDs into training and testing sets
train_match_ids, test_match_ids = train_test_split(match_ids, test_size=0.2, random_state=42)

# Step 3: Filter the original data for train and test sets
train_set = dataset[dataset['matchid'].isin(train_match_ids)]
test_set = dataset[dataset['matchid'].isin(test_match_ids)]

# Step 4: Display results
print(f"Number of matches in training set: {len(train_match_ids)}")
print(f"Number of matches in testing set: {len(test_match_ids)}")
print(f"Number of rows in training set: {len(train_set)}")
print(f"Number of rows in testing set: {len(test_set)}")


Number of matches in training set: 141262
Number of matches in testing set: 35316
Number of rows in training set: 542421
Number of rows in testing set: 135722


In [7]:
from collections import defaultdict
from sklearn.metrics import jaccard_score
import numpy as np

# Step 1: Infer duos (ADC-Support pairs) from the dataset
adc_to_supports = defaultdict(set)
support_to_adcs = defaultdict(set)

# Populate the mappings
for matchid, match_data in train_set.groupby('matchid'):
    for teamid, team_data in match_data.groupby('teamid'):
        # Find the ADC and Support for each team
        adc = team_data[team_data['position'] == 'ADC']['championid']
        support = team_data[team_data['position'] == 'SUPP']['championid']
        
        if not adc.empty and not support.empty:
            adc = adc.iloc[0]
            support = support.iloc[0]
            
            # Add the pairing to both mappings
            adc_to_supports[adc].add(support)
            support_to_adcs[support].add(adc)

# Step 2: Compute Jaccard similarity
def compute_jaccard_similarity(champ_id, target_mapping):
    similarities = {}
    target_set = target_mapping.get(champ_id, set())
    
    for other_champ, other_set in target_mapping.items():
        if other_champ != champ_id:
            intersection = len(target_set & other_set)
            union = len(target_set | other_set)
            jaccard_similarity = intersection / union if union > 0 else 0
            similarities[other_champ] = jaccard_similarity
    
    return similarities

# Step 3: Recommend champions based on Jaccard similarity
def recommend_duo_partner(champ_id, position, top_n=5):
    if position == 'ADC':
        similarities = compute_jaccard_similarity(champ_id, adc_to_supports)
    elif position == 'SUPP':
        similarities = compute_jaccard_similarity(champ_id, support_to_adcs)
    else:
        raise ValueError("Position must be 'ADC' or 'SUPP'")
    
    # Sort recommendations by similarity and return top_n
    recommended_partners = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return recommended_partners

# Example usage:
champion_id = 119  # Example: Champion ID for an ADC (e.g., Draven)
position = 'ADC'
recommendations = recommend_duo_partner(champion_id, position)

print(f"Recommendations for champion {champion_id} as a {position}:")
for partner, similarity in recommendations:
    print(f"Champion {partner} with similarity {similarity:.2f}")


Recommendations for champion 119 as a ADC:
Champion 222 with similarity 0.84
Champion 67 with similarity 0.84
Champion 236 with similarity 0.83
Champion 81 with similarity 0.82
Champion 21 with similarity 0.82


In [8]:
champs = pd.read_csv("/Users/froguro/.cache/kagglehub/datasets/paololol/league-of-legends-ranked-matches/versions/9/champs.csv")

In [9]:
from collections import defaultdict
import numpy as np

# Step 1: Infer duos (ADC-Support pairs) from the dataset
adc_to_supports = defaultdict(set)
support_to_adcs = defaultdict(set)

# Populate the mappings
for matchid, match_data in train_set.groupby('matchid'):
    for teamid, team_data in match_data.groupby('teamid'):
        # Find the ADC and Support for each team
        adc = team_data[team_data['position'] == 'ADC']['championid']
        support = team_data[team_data['position'] == 'SUPP']['championid']
        
        if not adc.empty and not support.empty:
            adc = adc.iloc[0]
            support = support.iloc[0]
            
            # Add the pairing to both mappings
            adc_to_supports[adc].add(support)
            support_to_adcs[support].add(adc)

# Create a dictionary mapping champion IDs to names
champion_id_to_name = dict(zip(champs['id'], champs['name']))

# Step 2: Compute Jaccard similarity between an ADC and all Supports
def recommend_duo_partner_for_adc(adc_champion_id, top_n=5):
    """
    Recommend the most compatible Supports for a given ADC based on Jaccard similarity.
    """
    recommendations = {}
    target_supports = adc_to_supports.get(adc_champion_id, set())
    
    for support, paired_adcs in support_to_adcs.items():
        intersection = len(target_supports & paired_adcs)
        union = len(target_supports | paired_adcs)
        jaccard_similarity = intersection / union if union > 0 else 0
        recommendations[support] = jaccard_similarity
    
    # Sort recommendations by similarity and return top N
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Step 3: Compute Jaccard similarity between a Support and all ADCs
def recommend_duo_partner_for_support(support_champion_id, top_n=5):
    """
    Recommend the most compatible ADCs for a given Support based on Jaccard similarity.
    """
    recommendations = {}
    target_adcs = support_to_adcs.get(support_champion_id, set())
    
    for adc, paired_supports in adc_to_supports.items():
        intersection = len(target_adcs & paired_supports)
        union = len(target_adcs | paired_supports)
        jaccard_similarity = intersection / union if union > 0 else 0
        recommendations[adc] = jaccard_similarity
    
    # Sort recommendations by similarity and return top N
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Example usage
adc_champion_id = 42  # Example ADC (e.g., Corki)
support_recommendations = recommend_duo_partner_for_adc(adc_champion_id)

print(f"Top Support recommendations for ADC {champion_id_to_name[adc_champion_id]}:")
for support, similarity in support_recommendations:
    print(f"Support Champion {champion_id_to_name[support]} with similarity {similarity:.2f}")

support_champion_id = 267  # Example Support (e.g., Nami)
adc_recommendations = recommend_duo_partner_for_support(support_champion_id)

print(f"\nTop ADC recommendations for Support {champion_id_to_name[support_champion_id]}:")
for adc, similarity in adc_recommendations:
    print(f"ADC Champion {champion_id_to_name[adc]} with similarity {similarity:.2f}")



Top Support recommendations for ADC Corki:
Support Champion Thresh with similarity 0.29
Support Champion Leona with similarity 0.25
Support Champion Nami with similarity 0.25
Support Champion Blitzcrank with similarity 0.25
Support Champion Lulu with similarity 0.23

Top ADC recommendations for Support Nami:
ADC Champion Caitlyn with similarity 0.38
ADC Champion Jinx with similarity 0.36
ADC Champion Xayah with similarity 0.36
ADC Champion Lucian with similarity 0.35
ADC Champion Miss Fortune with similarity 0.35


In [10]:
from collections import defaultdict
import pandas as pd

# Initialize a dictionary to track wins and total games for each ADC-Support duo
duo_stats = defaultdict(lambda: {"wins": 0, "total": 0})

# Populate duo stats
for matchid, match_data in dataset.groupby('matchid'):
    for teamid, team_data in match_data.groupby('teamid'):
        # Find the ADC and Support for each team
        adc = team_data[team_data['position'] == 'ADC']['championid']
        support = team_data[team_data['position'] == 'SUPP']['championid']
        win = team_data['win'].iloc[0]  # Team's win status
        
        if not adc.empty and not support.empty:
            adc = adc.iloc[0]
            support = support.iloc[0]
            
            # Update the duo's stats
            duo_stats[(adc, support)]["total"] += 1
            duo_stats[(adc, support)]["wins"] += win

# Step 2: Create a DataFrame from the duo stats
duo_data = []
for (adc, support), stats in duo_stats.items():
    winrate = stats["wins"] / stats["total"]
    duo_data.append({"ADC": adc, "Support": support, "Wins": stats["wins"], "Total Games": stats["total"], "Winrate": winrate})

duo_df = pd.DataFrame(duo_data)

# Display the resulting DataFrame
print(duo_df.head())

# Save the DataFrame to a CSV file for later use
duo_df.to_csv("adc_support_duos_with_winrate.csv", index=False)


   ADC  Support  Wins  Total Games   Winrate
0  119      267   511          967  0.528438
1  222      161   115          228  0.504386
2   69      115     0            1  0.000000
3   51       43  1794         3555  0.504641
4  119       40   961         1744  0.551032


In [11]:
# Step 1: Create a lookup for winrates from the duo DataFrame
duo_winrate_lookup = {
    (row['ADC'], row['Support']): row['Winrate']
    for _, row in duo_df.iterrows()
}

# Step 2: Evaluate recommendations on the test set
def evaluate_recommendations(test_data, recommendations_function, position):
    """
    Evaluate the winrates of recommended duos in the test set.

    Parameters:
        test_data (pd.DataFrame): Test dataset
        recommendations_function (callable): Function to generate recommendations
        position (str): Position to evaluate ('ADC' or 'SUPP')
    
    Returns:
        List of winrates for the recommended duos.
    """
    winrates = []

    for matchid, match_data in test_data.groupby('matchid'):
        for teamid, team_data in match_data.groupby('teamid'):
            if position == 'ADC':
                champion = team_data[team_data['position'] == 'ADC']['championid']
            elif position == 'SUPP':
                champion = team_data[team_data['position'] == 'SUPP']['championid']
            else:
                raise ValueError("Position must be 'ADC' or 'SUPP'")

            if not champion.empty:
                champion = champion.iloc[0]

                # Get recommendations for the champion
                recommendations = recommendations_function(champion)

                # Check winrates for recommended duos
                for recommended_partner, _ in recommendations:
                    duo = (champion, recommended_partner) if position == 'ADC' else (recommended_partner, champion)
                    winrate = duo_winrate_lookup.get(duo, None)
                    if winrate is not None:
                        winrates.append(winrate)

    return winrates

# Step 3: Evaluate winrates of ADC recommendations
adc_winrates = evaluate_recommendations(test_set, recommend_duo_partner_for_adc, position='ADC')

# Step 4: Evaluate winrates of Support recommendations
support_winrates = evaluate_recommendations(test_set, recommend_duo_partner_for_support, position='SUPP')

# Step 5: Print average winrates
import numpy as np
print(f"Average winrate of ADC recommendations: {np.mean(adc_winrates):.2f}")
print(f"Average winrate of Support recommendations: {np.mean(support_winrates):.2f}")


Average winrate of ADC recommendations: 0.51
Average winrate of Support recommendations: 0.51


In [37]:
adc_champion_id = 516  # Example ADC 
support_recommendations = recommend_duo_partner_for_adc(adc_champion_id)

print(f"Top Support recommendations for ADC {champion_id_to_name[adc_champion_id]}:")

for support, similarity in support_recommendations:
    print(f"Support Champion {champion_id_to_name[support]} with similarity {similarity:.2f}")

Top Support recommendations for ADC Ornn:
Support Champion Ziggs with similarity 0.00
Support Champion Karma with similarity 0.00
Support Champion Janna with similarity 0.00
Support Champion Nami with similarity 0.00
Support Champion Rakan with similarity 0.00
